# DortmundCrawler

This is a bot that scans the Dortmund Data Bank VLE Index for the supplied compound names and returns tabular VLE data in the form of arrays. To take a look at the website that it scrapes, click here: http://www.ddbst.com/en/EED/VLE/VLEindex.php

It returns 4 things:
1. The tabular data in the form of arrays
2. The constant values associated with the tables (like constant temperature or pressure)
3. The status of the table (whether the table is T-X-Y (status=1) or P-X-Y (status=0))
4. The switch variable which indicates whether the mole fraction data is with respect to the first or second compound

In [48]:
import requests
from bs4 import BeautifulSoup
import scipy as sci
import pandas as pd

First, we define some functions that make crawling the page easier and more systematic. The tables are located under headers named "Data Table" so if we locate these headers on the page, we can extract the tables below them.

In [49]:
#Get the value of the constant (Pressure or Temperature) above the table
def GetConstantValue(useful_header):
    useful_value=useful_header.find_previous("table")
    col=useful_value.find_all("td")
    constvalue=col[0].get_text()
    constvalue=float(constvalue)
    return constvalue

#Get the all the headers named "Data Table" on the page
def GetUsefulHeaders(soup):
    headers=soup.find_all("h4")
    useful_headers=[]
    for header in headers:
        text=header.get_text()
        if(text=="Data Table"):
            useful_headers.append(header)
    return useful_headers

#With reference to the headers extracted above, get the tables that follow them. 
def GetUsefulTables(useful_headers):
    number_of_tables=len(useful_headers)
    list_of_arrays=[]
    constvalues=sci.zeros(number_of_tables)
    stats=sci.zeros(number_of_tables)
    for i in range(1,number_of_tables):
        useful_header=useful_headers[i]
        useful_table=useful_header.find_next("table")
        rows=useful_table.find_all("tr")
        number_of_cols=len(rows[-1].find_all("td"))
        if(number_of_cols<3): #Skip tables with less than 3 columns (tables with only X-Y data)
            continue
        constvalues[i]=GetConstantValue(useful_header)
        stats[i]=GetTableStat(useful_header)
        arr=sci.zeros((len(rows),3))
        
        #Get values in data cells in the table
        row_counter=0
        for row in rows:
            cols=row.find_all("td")
            for cell in range(len(cols)):
                arr[row_counter,cell]=cols[cell].get_text()
            row_counter+=1
        
        #Delete the first and last rows (mole fraction 0 and 1) 
        arr=sci.delete(arr,(0),axis=0)
        if(arr[0,1]==0.):
            arr=sci.delete(arr,(0),axis=0)
        if(arr[-1,1]==1.):
            arr=sci.delete(arr,(-1),axis=0)
        list_of_arrays.append(arr)
    
    final_constvalues=constvalues[constvalues!=0]
    final_stats=stats[constvalues!=0]
    return [list_of_arrays,final_constvalues,final_stats]
  

#Get status of table, 1 for T-X-Y data and 2 for P-X-Y data
def GetTableStat(useful_header):
    useful_value=useful_header.find_previous("table")
    col=useful_value.find_all("td")
    check=col[1].get_text()
    if(check=="K"):
        return 0
    else:
        return 1

Next, let us define the main Crawl function that will request the url and parse the page using the module BeautifulSoup. All the functions defined above are called in this function. 

In [50]:
def Crawl(compound1,compound2):
    comp1=f"{compound1}"
    comp2=f"{compound2}"
    page=requests.get(f"http://www.ddbst.com/en/EED/VLE/VLE%20{comp1}%3B{comp2}.php")
    if(page.status_code==200): #status code 200 indicates that the url was requested successfully
        switch=0
    else:
        switch=1
        page=requests.get(f"http://www.ddbst.com/en/EED/VLE/VLE%20{comp2}%3B{comp1}.php")
    soup=BeautifulSoup(page.text,"html.parser")

    useful_headers=GetUsefulHeaders(soup)
    [tables,constantvalues,stats]=GetUsefulTables(useful_headers)
    return [tables,constantvalues,stats,switch]

Now that all the required functions have been defined, let us choose two compounds (p-Xylene and Acetone) and scrape their VLE data from the Dortmund Data Bank website using our crawler.

In [51]:
compound1="p-Xylene"
compound2="Acetone"

[tables,constantvalues,stats,switch]=Crawl(compound1,compound2) #Call the main crawler function
stats

array([ 0.,  0.])

Since both the stat values are 0, both the tables are P-X-Y. Therefore, the constant values are temperature values in K. 

In [52]:
constantvalues

array([ 313.15,  353.15])

Now let us nicely frame the tables in dataframes.

In [53]:
table_1=tables[0] #store the first table in table_1
df1=pd.DataFrame({"Pressure (kPa)":pd.Series(table_1[:,0]),"X":pd.Series(table_1[:,1]),"Y":pd.Series(table_1[:,2])})
print(f"The first table is the P-X-Y data at a constant temperature of {constantvalues[0]} K")
display(df1)

The first table is the P-X-Y data at a constant temperature of 313.15 K


,Pressure (kPa),X,Y
0,9.9,0.0473,0.6217
1,19.0,0.1427,0.8500
2,29.2,0.2943,0.9148
3,40.7,0.5735,0.9543
4,50.3,0.8680,0.9844
5,53.5,0.9486,0.9851


In [54]:
table_2=tables[1] #store the first table in table_2
df2=pd.DataFrame({"Pressure (kPa)":pd.Series(table_2[:,0]),"X":pd.Series(table_2[:,1]),"Y":pd.Series(table_2[:,2])})
print("The second table is the P-X-Y data at a constant temperature of {constantvalues[1]} K")
display(df2)

The second table is the P-X-Y data at a constant temperature of {constantvalues[1]} K


,Pressure (kPa),X,Y
0,33.600,0.0327,0.4535
1,75.400,0.1431,0.7640
2,144.700,0.5549,0.9342
3,171.101,0.7242,0.9567
4,188.101,0.8716,0.9794
5,205.400,0.9712,0.9943
